In [ ]:
import json
import numpy
import pandas
from pprint import pprint
pandas.options.display.max_colwidth = -1

def parse_events(thisgame_df):
    thisgame_df['play'] = thisgame_df['event'].str.split('/').str.get(0)
    thisgame_df['modifiers'] = thisgame_df['event'].str.split('/',n=1).str.get(1)
    thisgame_df['play'] = thisgame_df['event'].str.split('/').str.get(0)
    thisgame_df['modifiers'] = thisgame_df['event'].str.split('/',n=1).str.get(1)
    thisgame_df['modifiers'] = thisgame_df['modifiers'].fillna('')
    thisgame_df['baserunning'] = thisgame_df['modifiers'].str.split('.',n=1).str.get(1)
    thisgame_df['baserunning'] = thisgame_df['baserunning'].fillna('')
    thisgame_df['modifiers'] = thisgame_df['modifiers'].str.split('.').str.get(0)
    
    return thisgame_df

In [ ]:
# runs in about 5 minutes
with open('events_2012-2016.json','r') as f:
#with open('events_1927_1938.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')

In [ ]:
#allgames_df.sample(10).index
#mygames = ['NYN201204100', 'BAL201208060', 'SEA201208150', 'BAL201208300', 'BAL201209300', 'BAL201304210']
#mygames += ['BAL201305170', 'BAL201405010', 'BAL201406130', 'BAL201406250', 'MIA201407030', 'PIT201605180']
#mygames += ['SDN201606150', 'WAS201607200', 'BAL201608180']

#weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
#weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

#somecolumns = allgames_df.columns
#allgames2_df = pandas.DataFrame(columns=somecolumns)
#allgames2_df['id'] = ''
#allgames2_df = allgames2_df.set_index('id')
#allgames2_df
#for thisid, thisrow in allgames_df.iterrows():
#    if (thisid in mygames):
#        allgames2_df = allgames2_df.append(thisrow)
#allgames_df = allgames2_df
#print('done')

In [ ]:


column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_batting_order', 'sub_position']

allevents_df = pandas.DataFrame(columns = column_list)

cnt = 0


for thisgameID,row in allgames_df.iterrows():
    if (numpy.mod(cnt,100) == 0):
        print('Parsing game {0:,.0f}: {1}'.format(cnt, thisgameID))
    thisgame_df = pandas.DataFrame(row['events'], columns=column_list)
#    thisgame_df = thisgame_df.set_index('event_in_game')

    thisgame_df['gameID'] = thisgameID
    
    thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
    thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])
    thisgame_df = parse_events(thisgame_df)

    allevents_df = pandas.concat((allevents_df, thisgame_df))
    
    cnt = cnt + 1

allevents_df = allevents_df.reset_index(drop=True)
allevents_df.index.name = 'eventID'

write_columns = ['gameID']
write_columns += ['event_in_game', 'event_type', 'inning', 'home_or_visitor']
write_columns += ['batterID', 'pitch_count', 'pitch_sequence', 'event']
write_columns += ['play', 'modifiers', 'baserunning']
write_columns += ['comment', 'sub_batting_order', 'sub_position']

allevents_df = allevents_df[write_columns]

allevents_df.to_csv('events_parsed_me.csv', encoding='utf-8')
print('Done')
#allevents_df